In [ ]:
#Importing Necessary modules
import pandas as pd
import numpy as np
import sklearn as sk
import joblib
from google.colab import drive
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
# Mounting Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Reading Dataset
path = "/content/drive/MyDrive/capstone/merged_dataframe.csv"
df = pd.read_csv(filepath_or_buffer=path)
df.head()

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,Date,Time
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,0,WITHDRAWAL,2022-03-15,10:24:00
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,2022-03-15,10:24:00
2,DEPOSIT-CASH-9833,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,2022-03-15,10:24:00
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,2022-03-15,10:24:00
4,DEPOSIT-CHECK-9806,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,2022-03-15,10:24:00


In [ ]:
# Understanding Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2934716 entries, 0 to 2934715
Data columns (total 14 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Transaction_Id    object 
 1   Sender_Id         object 
 2   Sender_Account    object 
 3   Sender_Country    object 
 4   Sender_Sector     int64  
 5   Sender_lob        object 
 6   Bene_Id           object 
 7   Bene_Account      object 
 8   Bene_Country      object 
 9   USD_amount        float64
 10  Label             int64  
 11  Transaction_Type  object 
 12  Date              object 
 13  Time              object 
dtypes: float64(1), int64(2), object(11)
memory usage: 313.5+ MB


In [ ]:
# Understanding Dataset Columns
df.describe()

,Sender_Sector,USD_amount,Label
count,2.934716e+06,2.934716e+06,2934716.0
mean,2.068736e+04,5.995703e+02,0.5
std,1.523452e+04,5.276985e+02,0.5
min,0.000000e+00,0.000000e+00,0.0
25%,5.722000e+03,2.283000e+02,0.0
50%,2.361600e+04,4.855500e+02,0.5
75%,3.197400e+04,8.448301e+02,1.0
max,4.999900e+04,1.999894e+04,1.0


In [ ]:
# Visualizing Dataset Rows
df.head()

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,Date,Time
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,0,WITHDRAWAL,2022-03-15,10:24:00
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,2022-03-15,10:24:00
2,DEPOSIT-CASH-9833,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,2022-03-15,10:24:00
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,2022-03-15,10:24:00
4,DEPOSIT-CHECK-9806,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,2022-03-15,10:24:00


In [ ]:
# Adding Columns
df["Date"] = pd.to_datetime(df["Date"],format='mixed')
df["Time"] = pd.to_datetime(df["Time"])
df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["day"] = df["Date"].dt.day
df["hour"] = df["Time"].dt.hour
df["minute"] = df["Time"].dt.minute
df["second"] = df["Time"].dt.second
df["quarter"] = df["Date"].dt.quarter
df["day_of_week"] = df["Time"].dt.day_of_week
df["weekend"] = np.where(df["Time"].isin([5,6]),1,0)
df.drop(labels=["Time","Date"],axis=1,inplace=True)

<ipython-input-7-34a2a96fc639>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Time"] = pd.to_datetime(df["Time"])


In [ ]:
# Visualizing Dataset Rows With New Columns
df.head()

,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,...,Transaction_Type,year,month,day,hour,minute,second,quarter,day_of_week,weekend
0,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,...,WITHDRAWAL,2022,3,15,10,24,0,1,4,0
1,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,...,QUICK-PAYMENT,2022,3,15,10,24,0,1,4,0
2,DEPOSIT-CASH-9833,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9812,ACCOUNT-9826,USA,802.54,...,DEPOSIT-CASH,2022,3,15,10,24,0,1,4,0
3,PAY-CHECK-9832,JPMC-CLIENT-9812,ACCOUNT-9825,USA,38145,CCB,JPMC-CLIENT-9814,ACCOUNT-9824,USA,989.09,...,PAY-CHECK,2022,3,15,10,24,0,1,4,0
4,DEPOSIT-CHECK-9806,Unknown,Unknown,Unknown,0,Unknown,JPMC-CLIENT-9789,ACCOUNT-9800,USA,786.78,...,DEPOSIT-CHECK,2022,3,15,10,24,0,1,4,0


In [ ]:
# Checking if Transaction_Id is Unique
df["Transaction_Id"].value_counts()

Transaction_Id
EXCHANGE-10115                         1
12/04/2024_14_48_39_sdv_id_3225929     1
12/04/2024_14_48_39_sdv_id_3225786     1
12/04/2024_14_48_39_sdv_id_3225792     1
12/04/2024_14_48_39_sdv_id_3225795     1
                                      ..
PAY-BILL-1435078                       1
PAY-BILL-1428048                       1
PAY-CHECK-1435079                      1
WITHDRAWAL-1435907                     1
12/04/2024_14_48_39_sdv_id_10121521    1
Name: count, Length: 2934716, dtype: int64

In [ ]:
# Retrive the Columns in Dataset
df.columns

Index(['Transaction_Id', 'Sender_Id', 'Sender_Account', 'Sender_Country',
       'Sender_Sector', 'Sender_lob', 'Bene_Id', 'Bene_Account',
       'Bene_Country', 'USD_amount', 'Label', 'Transaction_Type', 'year',
       'month', 'day', 'hour', 'minute', 'second', 'quarter', 'day_of_week',
       'weekend'],
      dtype='object')

In [ ]:
# Retrive Count of Each Unique the Columns in Dataset
for columns in df.columns:
    print(f"========{columns}========")
    print(df[columns].value_counts())

========Transaction_Id========
Transaction_Id
EXCHANGE-10115                         1
12/04/2024_14_48_39_sdv_id_3225929     1
12/04/2024_14_48_39_sdv_id_3225786     1
12/04/2024_14_48_39_sdv_id_3225792     1
12/04/2024_14_48_39_sdv_id_3225795     1
                                      ..
PAY-BILL-1435078                       1
PAY-BILL-1428048                       1
PAY-CHECK-1435079                      1
WITHDRAWAL-1435907                     1
12/04/2024_14_48_39_sdv_id_10121521    1
Name: count, Length: 2934716, dtype: int64
========Sender_Id========
Sender_Id
Unknown                 370747
BILL-COMPANY-146291      90614
JPMC-CLIENT-10098         4019
JPMC-CLIENT-26424         3957
JPMC-CLIENT-23702         3768
                         ...  
BILL-COMPANY-1172259         1
BILL-COMPANY-1165271         1
COMPANY-1169435              1
COMPANY-1171147              1
BILL-COMPANY-2178781         1
Name: count, Length: 59072, dtype: int64
========Sender_Account========
Sender_Acco

In [ ]:
# Check For Null Values
df.isna().sum()

Transaction_Id      0
Sender_Id           0
Sender_Account      0
Sender_Country      0
Sender_Sector       0
Sender_lob          0
Bene_Id             0
Bene_Account        0
Bene_Country        0
USD_amount          0
Label               0
Transaction_Type    0
year                0
month               0
day                 0
hour                0
minute              0
second              0
quarter             0
day_of_week         0
weekend             0
dtype: int64

# Feature Engineering

In [ ]:
df["sender_type"] = df["Sender_Id"].apply(lambda sender_id: "-".join(sender_id.split("-")[:-1]) if "-" in sender_id else sender_id)
df.drop(labels="Sender_Id",axis=1,inplace=True)

In [ ]:
df["Bene_type"] = df["Bene_Id"].apply(lambda sender_id: "-".join(sender_id.split("-")[:-1]) if "-" in sender_id else sender_id)
df.drop(labels="Bene_Id",axis=1,inplace=True)

In [ ]:
# Counting Unique Values in sender_type, Bene_type
df["sender_type"].value_counts(),df["Bene_type"].value_counts()

(sender_type
 JPMC-CLIENT     2050385
 Unknown          370747
 CLIENT           219289
 BILL-COMPANY     129953
 COMPANY          112972
 JPMC-COMPANY      51370
 Name: count, dtype: int64,
 Bene_type
 JPMC-CLIENT     1039993
 BILL-COMPANY     750638
 CLIENT           607590
 Unknown          354955
 COMPANY          127825
 JPMC-COMPANY      53715
 Name: count, dtype: int64)

In [ ]:
# Visualize The dataset Rows
df.head()

,Transaction_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type,...,month,day,hour,minute,second,quarter,day_of_week,weekend,sender_type,Bene_type
0,EXCHANGE-10115,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,558.43,0,WITHDRAWAL,...,3,15,10,24,0,1,4,0,JPMC-CLIENT,Unknown
1,QUICK-PAYMENT-10116,ACCOUNT-10109,USA,15287,CCB,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT,...,3,15,10,24,0,1,4,0,JPMC-CLIENT,CLIENT
2,DEPOSIT-CASH-9833,Unknown,Unknown,0,Unknown,ACCOUNT-9826,USA,802.54,0,DEPOSIT-CASH,...,3,15,10,24,0,1,4,0,Unknown,JPMC-CLIENT
3,PAY-CHECK-9832,ACCOUNT-9825,USA,38145,CCB,ACCOUNT-9824,USA,989.09,0,PAY-CHECK,...,3,15,10,24,0,1,4,0,JPMC-CLIENT,JPMC-CLIENT
4,DEPOSIT-CHECK-9806,Unknown,Unknown,0,Unknown,ACCOUNT-9800,USA,786.78,0,DEPOSIT-CHECK,...,3,15,10,24,0,1,4,0,Unknown,JPMC-CLIENT


In [ ]:
# Dropping Unecassary Columns
df.drop(["Bene_Account","Sender_Account","Transaction_Id","year"],axis=1,inplace=True)

In [ ]:
# Checking Columns
df.columns

Index(['Sender_Country', 'Sender_Sector', 'Sender_lob', 'Bene_Country',
       'USD_amount', 'Label', 'Transaction_Type', 'month', 'day', 'hour',
       'minute', 'second', 'quarter', 'day_of_week', 'weekend', 'sender_type',
       'Bene_type'],
      dtype='object')

In [ ]:
# Counting Unique Values in Sender_Sector
df["Sender_Sector"].value_counts()

Sender_Sector
0        463062
49999      7945
24881       896
24904       879
24856       874
          ...  
42955        19
42012        18
49491        18
43729        18
23769        17
Name: count, Length: 50000, dtype: int64

# Encoding The Data

In [ ]:
Sender_lob_dummies  = pd.get_dummies(df["Sender_lob"],dtype="int")
df.drop("Sender_lob",axis=1,inplace=True)
df = df.assign(**Sender_lob_dummies)

In [ ]:
Bene_Country_label_encoder = LabelEncoder()
df['Bene_Country_Encoded'] = Bene_Country_label_encoder.fit_transform(df['Bene_Country'])
df.drop("Bene_Country",axis=1,inplace=True)

Sender_Country_label_encoder = LabelEncoder()
df['Sender_Country_Encoded'] = Sender_Country_label_encoder.fit_transform(df['Sender_Country'])
df.drop("Sender_Country",axis=1,inplace=True)

Transaction_Type_label_encoder = LabelEncoder()
df['Transaction_Type_Encoded'] = Transaction_Type_label_encoder.fit_transform(df['Transaction_Type'])
df.drop("Transaction_Type",axis=1,inplace=True)

sender_type_label_encoder = LabelEncoder()
df['sender_type_Encoded'] = sender_type_label_encoder.fit_transform(df['sender_type'])
df.drop("sender_type",axis=1,inplace=True)

Bene_type_label_encoder = LabelEncoder()
df['Bene_type_Encoded'] = Bene_type_label_encoder.fit_transform(df['Bene_type'])
df.drop("Bene_type",axis=1,inplace=True)

In [ ]:
scaler = StandardScaler()
scaled_values = scaler.fit_transform(df)
df= pd.DataFrame(scaled_values, columns=df.columns)

In [ ]:
# Visualizing The Data
df.head()

,Sender_Sector,USD_amount,Label,month,day,hour,minute,second,quarter,day_of_week,weekend,CCB,Unknown,Bene_Country_Encoded,Sender_Country_Encoded,Transaction_Type_Encoded,sender_type_Encoded,Bene_type_Encoded
0,0.974736,-0.077962,-1.0,-1.682708,0.580799,-0.53219,-0.990134,-2.4118,-2.048833,0.0,0.0,0.513734,-0.513734,0.690786,0.470886,1.806356,0.045161,1.789428
1,-0.354482,0.043983,-1.0,-1.682708,0.580799,-0.53219,-0.990134,-2.4118,-2.048833,0.0,0.0,0.513734,-0.513734,-1.865526,0.470886,1.313721,0.045161,-0.624836
2,-1.357927,0.384632,-1.0,-1.682708,0.580799,-0.53219,-0.990134,-2.4118,-2.048833,0.0,0.0,-1.946532,1.946532,0.665350,0.502561,-1.642089,1.829607,0.582296
3,1.145926,0.738148,-1.0,-1.682708,0.580799,-0.53219,-0.990134,-2.4118,-2.048833,0.0,0.0,0.513734,-0.513734,0.665350,0.470886,0.821086,0.045161,0.582296
4,-1.357927,0.354767,-1.0,-1.682708,0.580799,-0.53219,-0.990134,-2.4118,-2.048833,0.0,0.0,-1.946532,1.946532,0.665350,0.502561,-1.149454,1.829607,0.582296


In [ ]:
# Conunting labels in The Data
df["Label"].value_counts()

Label
-1.0    1467358
 1.0    1467358
Name: count, dtype: int64

In [ ]:
# Understanding Transformed Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2934716 entries, 0 to 2934715
Data columns (total 18 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Sender_Sector             float64
 1   USD_amount                float64
 2   Label                     float64
 3   month                     float64
 4   day                       float64
 5   hour                      float64
 6   minute                    float64
 7   second                    float64
 8   quarter                   float64
 9   day_of_week               float64
 10  weekend                   float64
 11  CCB                       float64
 12  Unknown                   float64
 13  Bene_Country_Encoded      float64
 14  Sender_Country_Encoded    float64
 15  Transaction_Type_Encoded  float64
 16  sender_type_Encoded       float64
 17  Bene_type_Encoded         float64
dtypes: float64(18)
memory usage: 403.0 MB


In [ ]:
# Didviding the data
X_data = df.drop(["Label"],axis=1)
Y_data =  df["Label"]

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2054301, 17), (880415, 17), (2054301,), (880415,))

# Machine Learning Models
## Supervised

In [ ]:
# # Support Vector Machine (SVM) Classifier
# from sklearn.svm import SVC
# svm_clf = SVC()
# svm_clf.fit(X_train, y_train)
# svm_pred = svm_clf.predict(X_test)
# svm_accuracy = accuracy_score(y_test, svm_pred)
# print("SVM Accuracy:", svm_accuracy)

In [ ]:
# # K-Nearest Neighbors (KNN) Classifier
# from sklearn.neighbors import KNeighborsClassifier
# knn_clf = KNeighborsClassifier()
# knn_clf.fit(X_train, y_train)
# knn_pred = knn_clf.predict(X_test)

In [ ]:
# knn_classification_report = classification_report(y_test, knn_pred)
# print(knn_classification_report)

In [ ]:
# Implementing the   decision tree model sklearn.tree.DecisionTreeClassifier

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

# Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Decision Tree Classifer Accuracy:", accuracy_score(y_test, y_pred))

Decision Tree Classifer Accuracy: 0.9772993417876796


In [ ]:
# Random Forest Classifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)

In [ ]:
rf_classification_report = classification_report(y_test, rf_pred)
print(rf_classification_report)

              precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99    440179
         1.0       1.00      0.98      0.99    440236

    accuracy                           0.99    880415
   macro avg       0.99      0.99      0.99    880415
weighted avg       0.99      0.99      0.99    880415



In [ ]:
# Logistic Regression Classifier
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)

In [ ]:
lr_classification_report = classification_report(y_test, lr_pred)
print(lr_classification_report)

              precision    recall  f1-score   support

        -1.0       0.98      0.96      0.97    440179
         1.0       0.96      0.98      0.97    440236

    accuracy                           0.97    880415
   macro avg       0.97      0.97      0.97    880415
weighted avg       0.97      0.97      0.97    880415



In [ ]:
# Multi-Layer Perceptron (MLP) Classifier
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
mlp_pred = mlp_clf.predict(X_test)

In [ ]:
mlp_classification_report = classification_report(y_test, mlp_pred)
print(mlp_classification_report)

              precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99    440179
         1.0       1.00      0.98      0.99    440236

    accuracy                           0.99    880415
   macro avg       0.99      0.99      0.99    880415
weighted avg       0.99      0.99      0.99    880415



In [ ]:
# AdaBoost Classifier
ada_clf = AdaBoostClassifier()
ada_clf.fit(X_train, y_train)
ada_pred = ada_clf.predict(X_test)

In [ ]:
ada_classification_report = classification_report(y_test, ada_pred)
print(ada_classification_report)

              precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99    440179
         1.0       1.00      0.98      0.99    440236

    accuracy                           0.99    880415
   macro avg       0.99      0.99      0.99    880415
weighted avg       0.99      0.99      0.99    880415



In [ ]:
# Naive Bayes Classifier (Gaussian)
nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train)
nb_pred = nb_clf.predict(X_test)

In [ ]:
nb_classification_report = classification_report(y_test, nb_pred)
print(nb_classification_report)

              precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99    440179
         1.0       1.00      0.98      0.99    440236

    accuracy                           0.99    880415
   macro avg       0.99      0.99      0.99    880415
weighted avg       0.99      0.99      0.99    880415



In [ ]:
# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)

In [ ]:
gb_classification_report = classification_report(y_test, gb_pred)
print(gb_classification_report)

              precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99    440179
         1.0       1.00      0.98      0.99    440236

    accuracy                           0.99    880415
   macro avg       0.99      0.99      0.99    880415
weighted avg       0.99      0.99      0.99    880415



In [ ]:
# Save the trained models
# joblib.dump(knn_clf, 'knn_model.pkl')
joblib.dump(clf, 'decision_tree_model.pkl')
joblib.dump(rf_clf, 'random_forest_model.pkl')
joblib.dump(lr_clf, 'logistic_regression_model.pkl')
joblib.dump(mlp_clf, 'mlp_model.pkl')
joblib.dump(ada_clf, 'adaboost_model.pkl')
joblib.dump(nb_clf, 'naive_bayes_model.pkl')
joblib.dump(gb_clf, 'gradient_boosting_model.pkl')

# Load the saved models
# loaded_knn_model = joblib.load('knn_model.pkl')
loaded_decision_tree_model = joblib.load('decision_tree_model.pkl')
loaded_random_forest_model = joblib.load('random_forest_model.pkl')
loaded_logistic_regression_model = joblib.load('logistic_regression_model.pkl')
loaded_mlp_model = joblib.load('mlp_model.pkl')
loaded_adaboost_model = joblib.load('adaboost_model.pkl')
loaded_naive_bayes_model = joblib.load('naive_bayes_model.pkl')
loaded_gradient_boosting_model = joblib.load('gradient_boosting_model.pkl')

# Test the loaded models
# KNN
# y_pred_knn = loaded_knn_model.predict(X_test)
# accuracy_knn = classification_report(y_test, y_pred_knn)
# print("KNN classification_report:", accuracy_knn)

# Decision Tree
y_pred_dt = loaded_decision_tree_model.predict(X_test)
accuracy_dt = classification_report(y_test, y_pred_dt)
print("Decision Tree classification_report:", accuracy_dt)

# Random Forest
y_pred_rf = loaded_random_forest_model.predict(X_test)
accuracy_rf = classification_report(y_test, y_pred_rf)
print("Random Forest classification_report:", accuracy_rf)

# Logistic Regression
y_pred_lr = loaded_logistic_regression_model.predict(X_test)
accuracy_lr = classification_report(y_test, y_pred_lr)
print("Logistic Regression classification_report:", accuracy_lr)

# MLP
y_pred_mlp = loaded_mlp_model.predict(X_test)
accuracy_mlp = classification_report(y_test, y_pred_mlp)
print("MLP classification_report:", accuracy_mlp)

# AdaBoost
y_pred_ada = loaded_adaboost_model.predict(X_test)
accuracy_ada = classification_report(y_test, y_pred_ada)
print("AdaBoost classification_report:", accuracy_ada)

# Naive Bayes
y_pred_nb = loaded_naive_bayes_model.predict(X_test)
accuracy_nb = classification_report(y_test, y_pred_nb)
print("Naive Bayes classification_report:", accuracy_nb)

# Gradient Boosting
y_pred_gb = loaded_gradient_boosting_model.predict(X_test)
accuracy_gb = classification_report(y_test, y_pred_gb)
print("Gradient Boosting classification_report:", accuracy_gb)


Decision Tree classification_report:               precision    recall  f1-score   support

        -1.0       0.98      0.97      0.98    440179
         1.0       0.97      0.98      0.98    440236

    accuracy                           0.98    880415
   macro avg       0.98      0.98      0.98    880415
weighted avg       0.98      0.98      0.98    880415

Random Forest classification_report:               precision    recall  f1-score   support

        -1.0       0.98      1.00      0.99    440179
         1.0       1.00      0.98      0.99    440236

    accuracy                           0.99    880415
   macro avg       0.99      0.99      0.99    880415
weighted avg       0.99      0.99      0.99    880415

Logistic Regression classification_report:               precision    recall  f1-score   support

        -1.0       0.98      0.96      0.97    440179
         1.0       0.96      0.98      0.97    440236

    accuracy                           0.97    880415
   macro av